In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/data')

In [0]:
%tensorflow_version 1.x
!pip install texar

     |████████████████████████████████| 296kB 5.1MB/s 
     |████████████████████████████████| 645kB 45.1MB/s 
     |████████████████████████████████| 17.4MB 127kB/s 
     |████████████████████████████████| 1.0MB 41.5MB/s 
  Created wheel for texar: filename=texar-0.2.4-cp36-none-any.whl size=444369 sha256=881bec5b973437fa5bf5863d59af52ef2016ed7a8370e1a90a41cf3025bcfd41
  Stored in directory: /root/.cache/pip/wheels/d3/10/64/40ab4b8563fe393750ada8484c27da019b15b0ff51ffcbaf95
Successfully built texar
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.4
    Uninstalling numpy-1.17.4:
      Successfully uninstalled numpy-1.17.4


In [0]:
from collections import Counter
import json
import random
import re
import texar.tf as tx

Online Collection Part

In [0]:
char_counter = Counter()

qa_pairs = set() # to remove duplicate dialogue pairs

with open('../data/raw_data/xiaohuangji50w_nofenci.conv') as f:
  xiaohuangji = f.read().split('E')

with open('../data/raw_data/stc-3_emotion_train.json') as f:
  
  for turn in xiaohuangji:
    li = turn.strip().split('M ')
    if len(li) == 3:
      q, a = li[1:]
      q = q.strip()
      a = a.strip()
      if a == '= =' or a == '=。=':
        continue
      
      qa_pairs.add((q, a))
      
      char_counter.update(list(q))
      char_counter.update(list(a))
  
  for li in json.load(f):
    if len(li) > 2:
      continue
    else:
      (q, _), (a, _) = li
      q, a = ''.join(q.split()), ''.join(a.split())
      
      qa_pairs.add((q, a))
      
      char_counter.update(list(q))
      char_counter.update(list(a))

with open('../data/full.txt', 'w') as f_out:
  for q, a in qa_pairs:
    f_out.write(q+'|'+a+'\n')
  
chars = ['<pad>', '<start>', '<end>'] + [char for char, freq in char_counter.most_common() if freq >= 5]
with open('../vocab/char.txt', 'w') as f:
  for c in chars:
    f.write(c+'\n')

Reduce Online Collection

In [0]:
with_scores = []
with open('full.txt') as f:
  for line in f:
    line = line.rstrip()
    sp = line.split('|')
    if len(sp) == 2:
      source, target = sp
    else:
      continue
    if source == target:
      continue
    if '笑话' in source:
      continue
    if '傻逼' in target:
      continue
    try:
      score = tx.evals.sentence_bleu(references=[list(source)], hypothesis=list(target), max_order=2)
    except:
      continue
    if score > 70.:
      continue
    if score == 0.:
      continue
    with_scores.append((source, target, score))

with_scores.sort(key=lambda x: x[2])
with open('bleu_score.txt', 'w') as f_bleu, open('reduced.txt', 'w') as f_reduced:
  for triple in reversed(with_scores):
    source, target, score = triple
    f_bleu.write(source+'|'+target+'|'+str(score)+'\n')
    f_reduced.write(source+'|'+target+'\n')

Human Labelled Part

In [0]:
with open('../data/raw_data/results1017.txt') as f, open('../data/core.txt', 'w') as f_out:
  for i, line in enumerate(f):
    if i == 0:
      continue
    line = line.rstrip()
    sp = line.split('\t')
    
    if len(sp) > 2:
      f_out.write(sp[0]+'|'+sp[2]+'\n')
    else:
      f_out.write(sp[0]+'|'+re.sub(r'\(.*\)\s', '', sp[1])+'\n')